# Computer Vision Assignment 2

## Read and prepate images

In [1]:

import skimage
from skimage import data, io, filters

import matplotlib.pyplot as plt

p = Path("./data/train/")
pattern = 'train_lbl*.png'
label_files = list(p.glob(pattern))

NameError: name 'Path' is not defined

In [ ]:
selection = '002_11'
image_file = p / ('train_img_' + selection + '.png')
label_file = p / ('train_lbl_' + selection + '.png')

image_file

image = skimage.io.imread(image_file)
label = skimage.io.imread(label_file)

plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(label)

print(label.shape)

In [3]:
# def gen_batch_binary(batch_size, height: int, width: int):
#     image_list = []
#     label_list = []
# #     for i in range(batch_size):
#     for i in range(3):
#         print(i)
# #         img, label = gen_random_binary_test_image(height=height, width=width)
#         img = skimage.io.imread(image_file)
#         label = skimage.io.imread(label_file)
#         image_list.append(img.astype(np.float32) / 255.0)
#         label_list.append(label.astype(np.float32))
        
#     print(label_list)

#     image_batch = np.array(image_list, dtype=np.float32)
#     label_batch = np.array(label_list, dtype=np.float32)

#     # reshape labels as this is not done in the model
#     label_batch = label_batch.reshape(batch_size, height, width, 1)

#     return image_batch, label_batch

In [4]:
# image_batch, label_batch = gen_batch_binary(100, 256, 256)
# print(image_batch.shape)
# print(label_batch.shape)

In [2]:
def _parse_function(image_filename, label_filename, channels: int):
    """
    Parse image and label and return them. The image is divided by 255.0 and returned as float,
    the label is returned as is in uint8 format.
    Args:
    image_filename: name of the image file
    label_filename: name of the label file
    channels: channels of the input image, (the label is always one channel)
    Returns:
    tensors for the image and label read operations
    """
    
    image_string = tf.io.read_file(image_filename)
    image_decoded = tf.image.decode_png(image_string, channels=channels)
    image_decoded = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)

    # normalize image to zero mean
    image = tf.multiply(image_decoded, 2.0)
    image = tf.subtract(image, 1.0)

    label_string = tf.io.read_file(label_filename)
    label = tf.image.decode_png(label_string, dtype=tf.uint8, channels=1)

    return image, label

In [13]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


file_type_str = "train"
path = Path("./data/train/")
channels = 3
patch_size_width = 256
patch_size_height = 256

image_files = list(path.glob(file_type_str+'_img*.png'))
label_files = list(path.glob(file_type_str+'_lbl*.png'))

# make sure they are in the same order
image_files.sort()
label_files.sort()

image_files_array = np.asarray([str(p) for p in image_files])
label_files_array = np.asarray([str(p) for p in label_files])

print(image_files_array)

dataset = tf.data.Dataset.from_tensor_slices((image_files_array, label_files_array))
# shuffle the filename, unfortunately, then we cannot cache them
dataset = dataset.shuffle(buffer_size=10000)
# read the images

dataset = dataset.map(lambda image, file: _parse_function(image, file, channels=channels))
# Set the sizes of the input image, as keras needs to know them and remove last dimension of label image

dataset = dataset.map(lambda x, y: 
                      (tf.reshape(x, shape=(patch_size_width, patch_size_height, channels)),
                       tf.reshape(y, shape=(patch_size_width, patch_size_height))))


tst_img, tst_label = tuple(dataset)[211]

plt.subplot(1,2,1)
plt.imshow(tst_img)
plt.subplot(1,2,2)
plt.imshow(tst_label)

['data/train/train_img_000_00.png' 'data/train/train_img_000_01.png'
 'data/train/train_img_000_02.png' ... 'data/train/train_img_121_29.png'
 'data/train/train_img_121_30.png' 'data/train/train_img_121_31.png']


AttributeError: 'MapDataset' object has no attribute 'size'

### Define fully connected Network

In [5]:
from tensorflow import keras

def fcn_simple(input_height:int, input_width:int) -> keras.Model:
    """
    Create a simple fcn model for semantic segmentation with 2 classes
    """
    model = keras.Sequential()
    
    # we use grayscale (1-channel input)
    
    # (used to define input shape on the first layers)
    model.add(keras.layers.Layer(input_shape=(input_height , input_width, 3)))

    # add 3 convolutional layers with 3x3 filters
    model.add(keras.layers.Convolution2D(filters=4, kernel_size=3, strides=(1, 1), padding='same', activation='relu'))
    model.add(keras.layers.Convolution2D(filters=4, kernel_size=3, strides=(1, 1), padding='same', activation='relu'))
    model.add(keras.layers.Convolution2D(filters=4, kernel_size=3, padding='same', activation='relu'))

    # go to logits which is the number of classes and add sigmoid layer for activation
    model.add(keras.layers.Convolution2D(filters=1, kernel_size=1, activation=None, 
                                         kernel_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.001, seed=None)))
    model.add(keras.layers.Activation('softmax'))
    
    # reshape so that we have a sample for each pixel
    model.add(keras.layers.Reshape(target_shape=(input_height, input_width, 1)))

    return model


In [10]:
PATCH_HEIGHT = 256
PATCH_WIDTH = 256

BATCH_SIZE = 50
EPOCHS = 50
STEPS_PER_EPOCHS = 10
model = fcn_simple(PATCH_HEIGHT, PATCH_WIDTH)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history  = model.fit(dataset, epochs=6, validation_split=0.1)

# history = model.fit_generator(
#         generator=batch_generator(BATCH_SIZE, PATCH_HEIGHT, PATCH_WIDTH),
#         epochs=EPOCHS,
#         verbose=1,
#         steps_per_epoch=STEPS_PER_EPOCHS,
#         shuffle=False,
#         callbacks=None)

Epoch 1/6


ValueError: in user code:

    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/compvis/lib/python3.8/site-packages/keras/engine/traini